In [1]:
import sys

sys.path =  ['/Users/Sebastian/Dropbox/_ot/code/mlxtend/'] + sys.path

In [2]:
from mlxtend.classifier import Perceptron

In [11]:
import string
import inspect
import os
import sys
import pkgutil
import shutil


def _obj_name(obj):
    if hasattr(obj, '__name__'):
        return obj.__name__

    
def docstring_to_markdown(docstring):
    new_docstring_lst = []
    
    for idx, line in enumerate(docstring.split('\n')):
        line = line.strip()
        if set(line) in ({'-'}, {'='}):
            new_docstring_lst[idx-1] = '*%s*' % new_docstring_lst[idx-1]
        new_docstring_lst.append(line)
        
    for idx, line in enumerate(new_docstring_lst[1:]):
        if line:
            if ' : ' in line:
                line = line.replace(' : ', '` : ')
                new_docstring_lst[idx+1] = '\n- `%s\n' % line
            elif not line.startswith('*'):
                new_docstring_lst[idx+1] = '   %s' % line.lstrip()
                
    clean_lst = []
    for line in new_docstring_lst:
        if set(line.strip()) not in ({'-'}, {'='}):
            clean_lst.append(line)
    return clean_lst    


def import_package(rel_path_to_package, package_name):
    try:
        curr_dir = os.path.dirname(os.path.realpath(__file__))
    except NameError:
        curr_dir = os.path.dirname(os.path.realpath(os.getcwd()))
    package_path = os.path.join(curr_dir, rel_path_to_package)
    if package_path not in sys.path:
        sys.path = [package_path] + sys.path
    package = __import__(package_name)
    return package

    
def get_subpackages(package):
    "importer, subpackage_name"
    return [i for i in pkgutil.iter_modules(package.__path__) if i[2]]
    
    
def get_modules(package):
    "importer, subpackage_name"
    return [i for i in pkgutil.iter_modules(package.__path__)]


def get_functions_and_classes(package):
    classes, functions = [], []
    for name, member in inspect.getmembers(package):
        if not name.startswith('_'):
            if inspect.isclass(member):
                classes.append([name, member])
            elif inspect.isfunction(member):
                functions.append([name, member])    
    return classes, functions


def generate_api_docs(package, api_dir, clean=True):
    prefix = package.__name__ + "."

    # clear the previous version
    if clean:
        if os.path.isdir(api_dir):
            shutil.rmtree(api_dir)

    # get subpackages
    for importer, pkg_name, is_pkg in pkgutil.iter_modules(package.__path__, prefix):
        if is_pkg:
            subpackage = __import__(pkg_name, fromlist="dummy")
            prefix = subpackage.__name__ + "."

            # get functions and classes
            classes, functions = get_functions_and_classes(subpackage)

            target_dir = os.path.join(api_dir, subpackage.__name__)

            # create the subdirs
            if not os.path.isdir(target_dir):
                os.makedirs(target_dir)

            # create markdown documents
            
            # class docs
            for cl in classes:
                with open(os.path.join(target_dir, cl[0]) + '.md', 'a') as f:
                    f.write('### %s\n\n' % cl[0])
                    class_doc = str(inspect.getdoc(cl[1]))
                    ds = docstring_to_markdown(class_doc)
                    formatted = '\n'.join(ds)
                    f.write(formatted)
                    
                    # class method docs
                    f.write('\n\n#### Methods\n\n')
                    members = inspect.getmembers(cl[1])
                    for m in members:
                        if not m[0].startswith('_'):
                            sig = str(inspect.signature(m[1])).replace('self, ', '')
                            f.write('\n\n##### %s%s\n\n' % (m[0], sig))
                            m_doc = docstring_to_markdown(str(inspect.getdoc(m[1])))
                            f.write('\n'.join(m_doc))

            # function docs
            for fn in functions:
                with open(os.path.join(target_dir, fn[0]) + '.md', 'a') as f:
                    f.write('### %s\n\n' % fn[0])
                    s = str(inspect.getdoc(fn[1]))
                    ds = docstring_to_markdown(s)
                    formatted = "\n".join(ds)
                    f.write(formatted)
                    
package = import_package('Dropbox/_ot/code/mlxtend/', 'mlxtend')
generate_api_docs(package=package, api_dir='../docs/sources/api', clean=True)

### Tests

In [231]:

inspect.getdoc(mlxtend.utils.counter.Counter)

"Class to display the progress of for-loop iterators.\n\nParameters\n----------\nstderr : bool (default: True)\n    Prints output to sys.stderr if True; uses sys.stdout otherwise.\nstart_newline : bool (default: True)\n    Prepends a new line to the counter, which prevents overwriting counters\n    if multiple counters are printed in succession.\n\nAttributes\n----------\ncurr_iter : int\n    The current iteration.\nstart_time : int\n    The system's time in seconds when the Counter was initialized."

In [219]:
inspect.signature(Perceptron.fit)

<Signature (self, X, y, init_weights=True)>

In [172]:
docstring_to_markdown(docstring)
inspect.getdoc(Perceptron)

NameError: name 'docstring' is not defined

In [173]:
import mlxtend
import_module('Dropbox/_ot/code/mlxtend/', 'mlxtend')
sub_pgk = get_subpackages(package=mlxtend)
for s in sub_pgk:
    import_module('Dropbox/_ot/code/mlxtend/', 'mlxtend.%s' % s[1])
    get_modules(s[1])

AttributeError: 'str' object has no attribute '__path__'

In [174]:
_obj_name(Perceptron)

'Perceptron'

In [175]:
inspect.getdoc(Perceptron)

"Perceptron classifier.\n\nParameters\n------------\neta : float (default: 0.1)\n    Learning rate (between 0.0 and 1.0)\nepochs : int (default: 50)\n    Number of passes over the training dataset.\nshuffle : bool (default: False)\n     Shuffles training data every epoch if True to prevent circles.\nrandom_seed : int\n    Random state for initializing random weights.\nzero_init_weight : bool (default: False)\n    If True, weights are initialized to zero instead of small random\n    numbers in the interval [-0.1, 0.1];\n    ignored if solver='normal equation'\n\nAttributes\n-----------\nw_ : 1d-array\n    Weights after fitting.\ncost_ : list\n    Number of misclassifications in every epoch."

In [176]:
print("\n".join(docstring_to_markdown(inspect.getdoc(Perceptron))))

Perceptron classifier.

***Parameters***

- `eta` : float (default: 0.1)

   Learning rate (between 0.0 and 1.0)

- `epochs` : int (default: 50)

   Number of passes over the training dataset.

- `shuffle` : bool (default: False)

   Shuffles training data every epoch if True to prevent circles.

- `random_seed` : int

   Random state for initializing random weights.

- `zero_init_weight` : bool (default: False)

   If True, weights are initialized to zero instead of small random
   numbers in the interval [-0.1, 0.1];
   ignored if solver='normal equation'

***Attributes***

- `w_` : 1d-array

   Weights after fitting.

- `cost_` : list

   Number of misclassifications in every epoch.


In [177]:
import_module('Dropbox/_ot/code/mlxtend/', 'mlxtend')
import mlxtend

In [178]:
inspect.getmembers(mlxtend.classifier)

[('Adaline', mlxtend.classifier.adaline.Adaline),
 ('EnsembleVoteClassifier',
  mlxtend.classifier.ensemble_vote.EnsembleVoteClassifier),
 ('LogisticRegression',
  mlxtend.classifier.logistic_regression.LogisticRegression),
 ('NeuralNetMLP', mlxtend.classifier.neuralnet_mlp.NeuralNetMLP),
 ('Perceptron', mlxtend.classifier.perceptron.Perceptron),
 ('__all__',
  ['Perceptron',
   'Adaline',
   'LogisticRegression',
   'NeuralNetMLP',
   'EnsembleVoteClassifier']),
 ('__builtins__',
  {'ArithmeticError': ArithmeticError,
   'AssertionError': AssertionError,
   'AttributeError': AttributeError,
   'BaseException': BaseException,
   'BlockingIOError': BlockingIOError,
   'BrokenPipeError': BrokenPipeError,
   'BufferError': BufferError,
   'BytesWarning': BytesWarning,
   'ChildProcessError': ChildProcessError,
   'ConnectionAbortedError': ConnectionAbortedError,
   'ConnectionError': ConnectionError,
   'ConnectionRefusedError': ConnectionRefusedError,
   'ConnectionResetError': Connectio

In [179]:
classes = []
functions = []

for name, member in inspect.getmembers(mlxtend.classifier):
    if not name.startswith('_'):
        if inspect.isclass(member):
            classes.append([name, member])
        elif inspect.isfunction(member):
            functions.append([name, member])
            
classes

[['Adaline', mlxtend.classifier.adaline.Adaline],
 ['EnsembleVoteClassifier',
  mlxtend.classifier.ensemble_vote.EnsembleVoteClassifier],
 ['LogisticRegression',
  mlxtend.classifier.logistic_regression.LogisticRegression],
 ['NeuralNetMLP', mlxtend.classifier.neuralnet_mlp.NeuralNetMLP],
 ['Perceptron', mlxtend.classifier.perceptron.Perceptron]]

## New

In [47]:
def list_modules(module):
    """Iterate through a list of variables define in a module's
    public namespace."""
    comps = [i for i in dir(module)]
    return comps
    #getattr(mod, var)
        
list_modules(mlxtend.classifier)

['Adaline',
 'EnsembleVoteClassifier',
 'LogisticRegression',
 'NeuralNetMLP',
 'Perceptron',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'adaline',
 'ensemble_vote',
 'logistic_regression',
 'neuralnet_mlp',
 'perceptron']

[(FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend/classifier'),
  'adaline',
  False),
 (FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend/classifier'),
  'ensemble_vote',
  False),
 (FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend/classifier'),
  'logistic_regression',
  False),
 (FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend/classifier'),
  'neuralnet_mlp',
  False),
 (FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend/classifier'),
  'perceptron',
  False)]

In [61]:
import pkgutil



get_subpackages(mlxtend)

[(FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend'),
  'classifier',
  True),
 (FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend'),
  'data',
  True),
 (FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend'),
  'evaluate',
  True),
 (FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend'),
  'feature_selection',
  True),
 (FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend'),
  'file_io',
  True),
 (FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend'),
  'general_plotting',
  True),
 (FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend'),
  'math',
  True),
 (FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend'),
  'preprocessing',
  True),
 (FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend'),
  'regression_utils',
  True),
 (FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend'),
  'regressor',
  True),
 (FileFinder('/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend'),
  'text',

In [66]:
__import__('mlxtend.text')

<module 'mlxtend' from '/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend/__init__.py'>

In [67]:
mlxtend.text

<module 'mlxtend.text' from '/Users/Sebastian/Dropbox/_ot/code/mlxtend/mlxtend/text/__init__.py'>